In [1]:
import numpy as np
from scipy.linalg import *
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame,Series
import math
import time

In [2]:
k = 230 #W/m°C 
L = 0.050 #m
t = 0.002 #m
h = 210 #W/m²°C Convecção Forçada
T0 = 80 #°C
T_infinito = 25 #°C
w = 1 #m

In [3]:
meshx = 10
deltax = L/(meshx-0.5)

meshy = 10
deltay = (t/2)/(meshy-1)

In [4]:
#Numérico 2D
tempo_inicial = time.time()
Matrix_coef_a = np.zeros((meshx*meshy,meshx*meshy))
Matrix_coef_b = np.zeros((meshx*meshy,1))
T=np.zeros((meshx*meshy,1))
ae = np.zeros((meshx,meshy))
aw = np.zeros((meshx,meshy))
an = np.zeros((meshx,meshy))
aS = np.zeros((meshx,meshy))
ap = np.zeros((meshx,meshy))
b = np.zeros((meshx,meshy))
 
for x in range(meshx):
    for y in range(meshy):
        #volume inferior esquerdo
        if (x == 0) and (y==0):
            aw[x,y] = 0
            ae[x,y] = k*(deltay/2)*w/deltax
            an[x,y] = k*deltax*w/deltay
            aS[x,y] = 0
            aw_f = k*(deltay/2)*w/(deltax/2)
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] +aw_f
            b[x,y] = aw_f*T0
            
        #volume inferior direito
        if (x == 0) and (y==meshy-1):
            aw[x,y] = k*(deltay/2)*w/deltax
            ae[x,y] = 0
            an[x,y] = k*(deltax/2)*w/deltay
            aS[x,y] = 0            
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] 
            b[x,y] = 0
        
        #volume superior esquerdo
        if (x == meshx-1) and (y==0):
            aw[x,y] = 0
            ae[x,y] = k*(deltay/2)*w/deltax
            an[x,y] = 0
            aS[x,y] = k*deltax*w/deltay
            an_f = h*deltax*w
            aw_f = k*(deltay/2)*w/(deltax/2)
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] +aw_f + an_f
            b[x,y] = aw_f*T0 + an_f*T_infinito
            
        #volume superior direito
        if (x == meshx-1) and (y==meshy-1):
            aw[x,y] = k*(deltay/2)*w/deltax
            ae[x,y] = 0
            an[x,y] = 0
            aS[x,y] = k*(deltax/2)*w/(deltay) 
            an_f = h*(deltax/2)*w
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] +an_f
            b[x,y] = an_f*T_infinito
            
        #volumes na face esquerda
        if (y == 0) and (x>0) and (x<meshx-1):
            aw[x,y] = 0
            ae[x,y] = k*deltay*w/deltax
            an[x,y] = k*deltax*w/deltay
            aS[x,y] = k*deltax*w/deltay
            aw_f = k*deltay*w/(deltax/2)
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] +aw_f
            b[x,y] = aw_f*T0
            
        #volumes na face direita
        if (y == meshy-1) and (x>0) and (x<meshx-1):
            aw[x,y] = k*deltay*w/deltax
            ae[x,y] = 0
            an[x,y] = k*(deltax/2)*w/deltay
            aS[x,y] = k*(deltax/2)*w/deltay            
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] 
            b[x,y] = 0  
            
        #volumes no norte
        if (x == meshx-1) and (y>0) and (y<meshy-1):
            aw[x,y] = k*(deltay/2)*w/deltax
            ae[x,y] = k*(deltay/2)*w/deltax
            an[x,y] = 0
            aS[x,y] = k*deltax*w/deltay 
            an_f = h*deltax*w
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] +an_f
            b[x,y] = an_f*T_infinito
            
        #volumes no sul
        if (x == 0) and (y>0) and (y<meshy-1):
            aw[x,y] = k*(deltay/2)*w/deltax
            ae[x,y] = k*(deltay/2)*w/deltax
            an[x,y] = k*deltax*w/deltay
            aS[x,y] = 0            
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] 
            b[x,y] = 0
            
        #volumes internos
        if (x>0) and (x < meshx-1) and (y>0) and (y<meshy-1):
            aw[x,y] = k*deltay*w/deltax
            ae[x,y] = aw[x,y]
            an[x,y] = k*deltax*w/deltay
            aS[x,y] = an[x,y]            
            ap[x,y] = aw[x,y] + ae[x,y] + an[x,y] + aS[x,y] 
            b[x,y] = 0
        
                     
#montando a matriz de coeficientes:
cont = 0
for x in range(meshx):
    for y in range(meshy):
        Matrix_coef_a[cont,cont] = ap[x,y]
        Matrix_coef_a[cont,cont-1] = -aw[x,y]
        
        if cont < meshx*meshy-1:
            Matrix_coef_a[cont,cont+1] = -ae[x,y]
        if cont < meshx*meshy-meshy:   
            Matrix_coef_a[cont,cont+meshy] = -an[x,y]
        if cont >meshy-1:
            Matrix_coef_a[cont,cont-meshy] = -aS[x,y]
        
        
        Matrix_coef_b[cont,0] = b[x,y]
        cont += 1
        
    
T = solve(Matrix_coef_a,Matrix_coef_b)

T_plot = np.zeros((meshx,meshy))

#transformando o vetor Temperatura em Matriz

cont = 0
for x in range(meshx):
    for y in range(meshy):
        T_plot[x,y] = T[cont]
        cont += 1

q_cond =np.zeros(meshx)
q_cond[0] = (T0 - T_plot[0,0]) * k *(deltay/2)*w/(deltax/2)
for x in range(1,meshx-1):
    q_cond[x] = (T0 - T_plot[x,0]) * k *deltay*w/(deltax/2)
q_cond[meshx-1] = (T0 - T_plot[meshx-1,0]) * k *(deltay/2)*w/(deltax/2)

q_conv = np.zeros(meshy)
for y in range(meshy-1):
    q_conv[y] = (T_plot[meshx-1,y] - T_infinito) * h * deltax*w
q_conv[meshy-1] = (T_plot[meshx-1,meshy-1] - T_infinito) * h * (deltax/2)*w

q_cond_total = sum(q_cond) *2
q_conv_total = sum(q_conv) *2

tempo_final = time.time()

tempo = tempo_final-tempo_inicial
